In [1]:
# import dependencies
import datetime as dt
import pandas as pd
import requests, io, zipfile, os

In [2]:
# format of url = "https://s3.amazonaws.com/tripdata/201306-citibike-tripdata.zip"
url_base = 'https://s3.amazonaws.com/tripdata/'

root = 'Resources'

result_files = []
rename_files = []

In [3]:
# files set up by month, get by 4 digit year and 2 digit month
d = dt.date.today()
# url_year = d.year
# url_month = d.month

url_year = 2013
url_month = 7
url_file_prefix = f'{url_year:04d}{url_month:02d}'
new_file = url_file_prefix + '.csv'
# url_file_prefix

# directory files same through 2016, changed for 2017
url_file_2016 = '-citibike-tripdata.zip'
url_file_2017 = '-citibike-tripdata.csv.zip'

# do not save folders
substring = 'citibike-tripdata.csv'

# build the proper url to get data from files in the directory
if url_year >= 2017:
    url = url_base + url_file_prefix + url_file_2017
else:
    url = url_base + url_file_prefix + url_file_2016
print(url) 

r = requests.get(url)
un_zipped = zipfile.ZipFile(io.BytesIO(r.content))

with un_zipped as zf:
    for file in zf.namelist():
        print(file)
        # not in root directory, pass, otherwise save
        if file.startswith('_'):
            pass
        else:
            zf.extract(file, root)                        
            result_files.append(file)
            rename_files.append(new_file)


#    ## WORKING CODE
# r = requests.get(url)
# un_zipped = zipfile.ZipFile(io.BytesIO(r.content))

# with un_zipped as zf:
#     for file in zf.namelist():
#         print(file)
#         # not in root directory, pass, otherwise save
#         if file.startswith('_'):
#             pass
#         else:
#             zf.extract(file, root)
#             result_files.append(result_file)
##STOP         
                
# with un_zipped as zf:
#     for file in zf.namelist():
#         print(file)
#         if file.startswith('20'):
#             zf.extract(file, root)
#             result_files.append(result_file)


# # unzipfolder.extract(file, root)
# for file in unzipfolder:
#     if file.endswith('.csv'):
#         unzipfolder.extract(file, root)
#         print(file)
#         result_files.append(file)

# for file in os.listdir(root):
#     if os.path.isfile(os.path.join(root, file)):
#         result_file = os.path.join(root, file)
#         if result_file.endswith('.csv'):
#             print(result_file)
#             result_files.append(result_file)

https://s3.amazonaws.com/tripdata/201307-citibike-tripdata.zip
2013-07 - Citi Bike trip data.csv


In [12]:
# Function to rename files for clarity and easier looping
def main():
    for i, file in enumerate(os.listdir(root)):
        print(root)
        source = f'{root}/{result_files[i]}'
        destination = f'{root}/{rename_files[i]}'
        os.rename(source, destination)
# Driver Code
if __name__ == '__main__':     
    # Calling main() function
    main()

Resources


In [4]:
result_files



['2013-07 - Citi Bike trip data.csv']

In [5]:
rename_files

['201307.csv']

In [ ]:
result_files

In [ ]:
df2 = pd.read_csv(result_files[0])
df2

In [ ]:
# for result in result_files:
#     df2 = pd.read_csv(url2, compression = 'zip', engine ='python')

In [ ]:
df3 = pd.read_csv(result_files[1])
df3

In [ ]:

# import required modules
import pandas as pd
import numpy as np
import time
  
# time taken to read data
s_time_chunk = time.time()
chunk = pd.read_csv('gender_voice_dataset.csv', chunksize=1000)
e_time_chunk = time.time()
  
print("With chunks: ", (e_time_chunk-s_time_chunk), "sec")
df = pd.concat(chunk)
  
# data
df.sample(10)

In [ ]:
## Using dask docomentation/code from https://www.geeksforgeeks.org/working-with-large-csv-files-in-python/
# import required modules
import pandas as pd
import numpy as np
import time
from dask import dataframe as df1
  
# time taken to read data
s_time_dask = time.time()
dask_df = df1.read_csv('gender_voice_dataset.csv')
e_time_dask = time.time()
  
print("Read with dask: ", (e_time_dask-s_time_dask), "seconds")
  
# data
dask_df.head(10)